<a href="https://colab.research.google.com/github/DhawaDong/ConvsNet-DDSM/blob/main/MyConvResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython import get_ipython
get_ipython().magic('reset -sf')
#%reset

In [ ]:
import pandas as pd;
from imutils import paths
import os
import math
import numpy as np
import joblib
from imutils import paths
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

In [ ]:
import torch
import random
import albumentations
import matplotlib.pyplot as plt
import argparse
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
#from utils import EarlyStopping, LRScheduler
#import openpyxl;
from PIL import Image
from tqdm import tqdm
import torchvision
from torchvision import models as models
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
from torchsummary import summary

In [ ]:
#MetaData for input images.....
#ImagePaths = pd.read_excel('/content/drive/MyDrive/ImageFolderCalcMass.xlsx');
#ImagePaths = pd.read_excel('/content/drive/MyDrive/ImageFolderCalcMassNoBlank.xlsx');
#Imageinput ant classes....
#ImagePaths["Pathology"].replace({"BENIGN": 0, "BENIGN_WITHOUT_CALLBACK": 0, "MALIGNANT": 1}, inplace = True)


In [ ]:
#df.loc[df['column_name'] == some_value]
#SampleSpace = ImagePaths.loc[ImagePaths['Pathology'] == 1]
#Sampled_Data = SampleSpace.sample(n=650, replace = False)
#NewTrainImagePaths= ImagePaths.append(Sampled_Data, ignore_index=True)   #df1.append(df2, ignore_index=True)
#NewTrainImagePaths = NewTrainImagePaths.sample(frac = 1)
#X = NewTrainImagePaths.ImagePath.values
#y = NewTrainImagePaths.Pathology.values

In [ ]:
#X = ImagePaths.ImagePath.values
#y = ImagePaths.Pathology.values

In [ ]:
#(xtrain, xtest, ytrain, ytest) = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
#TrainSetDataFrame = pd.DataFrame({'ImagePath': xtrain, 'Pathology': ytrain})
#TestSetDataFrame = pd.DataFrame({'ImagePath': xtest, 'Pathology': ytest})

In [ ]:
#TrainSetDataFrame.to_csv('/content/drive/MyDrive/TrainSetNew.csv')
#TestSetDataFrame.to_csv('/content/drive/MyDrive/TestSetNew.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
TrainDataFrame = pd.read_csv('/content/drive/MyDrive/TrainSetNew.csv')
TestDataFrame = pd.read_csv('/content/drive/MyDrive/TestSetNew.csv')

#TrainSet DataFrame
xtrain = TrainDataFrame.ImagePath.values
ytrain = TrainDataFrame.Pathology.values
#TestSet DataFrame
xtest = TestDataFrame.ImagePath.values
ytest = TestDataFrame.Pathology.values

In [ ]:
#frequency of class in TrainSet
(unique, counts) = np.unique(ytrain, return_counts=True)
Trainfrequencies = np.asarray((unique, counts)).T
Trainfrequencies

array([[   0, 2576],
       [   1, 2375]])

In [ ]:
#frequency of class in TestSet
(unique, counts) = np.unique(ytest, return_counts=True)
Testfrequencies = np.asarray((unique, counts)).T
Testfrequencies

array([[   0, 1080],
       [   1, 1043]])

In [ ]:
y = np.concatenate((ytrain, ytest))
#print(ytrain.size, ytest.size)

In [ ]:
labels = y
# one hot encode
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
#print(labels)
# pickle the label binarizer
joblib.dump(lb, 'lb.pkl')
print('Save the one-hot encoded binarized labels as a pickled file.')
#print(labels)

Save the one-hot encoded binarized labels as a pickled file.


In [ ]:
''' SEED Everything '''
def seed_everything(SEED=42):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True
SEED=42
seed_everything(SEED=SEED)
''' SEED Everything '''
# set computation device
device = ('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Computation device: {device}")

Computation device: cuda:0


In [ ]:
import sys
sys.argv=['']
del sys

In [ ]:
# image dataset module
class DDSMimageDataset(Dataset):
    def __init__(self, path, labels, tfms=None):
        self.X = path
        self.y = labels
        # apply augmentations
        if tfms == 0:  # if validating
          self.aug = albumentations.Compose([
                albumentations.Resize(224, 224, always_apply=True),    #224, 224,
                albumentations.Normalize(mean=[0.485], std=[0.229], always_apply=True)
            ])  #0.485, 0.456, 0.406;  0.229, 0.224, 0.225
        else: # if training
          self.aug = albumentations.Compose([
                albumentations.Resize(224, 224, always_apply=True),
                albumentations.HorizontalFlip(p=1.0),
                albumentations.ShiftScaleRotate(shift_limit=0.3,scale_limit=0.3,rotate_limit=30,p=1.0),
                albumentations.Normalize(mean=[0.485], std=[0.229], always_apply=True)
            ])  #0.485, 0.456, 0.406;  0.229, 0.224, 0.225
    def __len__(self):
        return (len(self.X))

    def __getitem__(self, i):
      image = Image.open(self.X[i])
      #image = image.convert('RGB')
      #image = self.aug(image=np.array(image))['image']
      #image = np.transpose(image, (2, 0, 1)).astype(np.float32)
      #label = self.y[i]

      image = self.aug(image=np.array(image))['image']
      #image = torch.from_numpy(image) #.unsqueeze(0)
      #image = np.transpose(image, (2, 0, 1)).astype(np.float32)
      label = self.y[i]
      return torch.tensor(image[None, ...], dtype=torch.float), torch.tensor(label, dtype=torch.long)
      #return torch.tensor(image, dtype=torch.float), torch.tensor(label, dtype=torch.long)

In [ ]:
BatchSize = 64 #32 #64 128;
#train_data = DDSMimageDataset(xtrain, ytrain, tfms=1)
test_data = DDSMimageDataset(xtest, ytest, tfms=0)

In [ ]:
# dataloaders
#trainloader = DataLoader(train_data, batch_size=BatchSize, shuffle=True)
testloader = DataLoader(test_data, batch_size=BatchSize, shuffle=False)

In [ ]:
def NewTrainLoader(BatchSize):
  train_data = DDSMimageDataset(xtrain, ytrain, tfms=1)
  trainloader = DataLoader(train_data, batch_size=BatchSize, shuffle=True)
  return train_data, trainloader

In [ ]:
train_data, trainloader = NewTrainLoader(BatchSize)

In [ ]:
# load the binarized labels
print('Loading label binarizer...')
lb = joblib.load('lb.pkl')
print(lb.classes_)

Loading label binarizer...
[0 1]


In [ ]:
class BuildingBlock(nn.Module):
    def __init__(self, in_channels, first_superblock, stride=1, internal_block=False):
      super(BuildingBlock, self).__init__()
      if first_superblock == True:
        if internal_block == False:
          self.layer_a = nn.Conv2d(in_channels, out_channels=in_channels, kernel_size=1, bias=False)
          self.residual_downsample = nn.Sequential(nn.Conv2d(in_channels, in_channels*4, kernel_size=1, bias=False),
                                                     nn.BatchNorm2d(in_channels*4))
        else:
          self.layer_a = nn.Conv2d(in_channels*4, out_channels=in_channels, kernel_size=1, bias=False)
          self.residual_downsample = nn.Identity()
      else:
        if internal_block == False:
          self.layer_a = nn.Conv2d(in_channels*2, out_channels=in_channels, kernel_size=1, stride=stride, bias=False)
          self.residual_downsample= nn.Sequential(nn.Conv2d(in_channels*2, out_channels=in_channels*4, kernel_size=1, stride=stride, bias=False),
                                                        nn.BatchNorm2d(in_channels*4))
        else:
            self.layer_a = nn.Conv2d(in_channels*4, out_channels=in_channels, stride=1, kernel_size=1, bias=False)
            self.residual_downsample = nn.Identity()

      self.bn_a = nn.BatchNorm2d(in_channels)
      self.layer_b = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, bias=False)
      self.bn_b = nn.BatchNorm2d(in_channels)
      self.layer_c = nn.Conv2d(in_channels, in_channels*4, kernel_size=1, bias=False)
      self.bn_c = nn.BatchNorm2d(in_channels * 4)
      self.relu = nn.ReLU(inplace=True)
      #self.relu = nn.Tanh(inplace=True)

    def forward(self, x):
      output = self.layer_a(x)
      output = self.bn_a(output)
      output = self.relu(output)
      #output = self.Tanh(output)

      output = self.layer_b(output)
      output = self.bn_b(output)
      output = self.relu(output)
      #output = self.Tanh(output)

      output = self.layer_c(output)
      output = self.bn_c(output)

      residual = x
      residual = self.residual_downsample(residual)
      output += residual

      output = self.relu(output)
      #output = self.Tanh(output)
      return output

In [ ]:
class ResNet(nn.Module):
  def __init__(self, building_block, num_layers, num_classes=2):
    super(ResNet, self).__init__()
    BatchSizeNet = BatchSize #64 #32 #64 128
    LinearOneOut = 2*2048 #64 #32;
    NumLayers = len(num_layers)-1
    self.internal_channels = BatchSizeNet
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=self.internal_channels, kernel_size=7,
                           stride=2, padding=3, bias=False)
    self.bn1 = nn.BatchNorm2d(self.internal_channels)
    self.relu = nn.ReLU(inplace=True)
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    #self.superblock2 = self.make_superblock(building_block, 64, num_layers[0], first_superblock=True)
    #self.superblock3 = self.make_superblock(building_block, 128, num_layers[1], stride=2)
    #self.superblock4 = self.make_superblock(building_block, 256, num_layers[2], stride=2)
    #self.superblock5 = self.make_superblock(building_block, 512, num_layers[3], stride=2)

    self.superblock2 = self.make_superblock(building_block, BatchSizeNet, num_layers[0], first_superblock=True)
    self.superblock3 = self.make_superblock(building_block, 2*BatchSizeNet, num_layers[1], stride=2)
    self.superblock4 = self.make_superblock(building_block, 4*BatchSizeNet, num_layers[2], stride=2)
    self.superblock5 = self.make_superblock(building_block, 8*BatchSizeNet, num_layers[3], stride=2)

    self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
    #self.avgpool = nn.MaxPool2d(2,2) #nn.AdaptiveAvgPool2d((1, 1))
    #self.fc1 = nn.Linear(512*4, 64)
    #self.fc = nn.Linear((4*(2**NumLayers)*BatchSizeNet), 2)
    self.fc1 = nn.Sequential(nn.Linear((4*(2**NumLayers)*BatchSizeNet), LinearOneOut), nn.BatchNorm1d(LinearOneOut), nn.ReLU(inplace = True))
    #self.fc2 = nn.Linear(256, 2)
    self.fc2 =  nn.Sequential(nn.Linear(LinearOneOut, 2)) #, nn.LogSoftmax(dim=1) )
    self.init_weights()

  def make_superblock(self, building_block, in_channels, num_blocks, stride=1, first_superblock=False,
                        internal_block=True):
    if first_superblock == True:
      layers = [building_block(in_channels, first_superblock)]
      for i in range(num_blocks-1):
        layers.append(building_block(in_channels, first_superblock, internal_block=True))
    else:
      layers = [building_block(in_channels, stride=stride, first_superblock=False, internal_block=False)]
      for i in range(num_blocks-1):
        layers.append(building_block(in_channels, first_superblock=False, internal_block=True))

    return nn.Sequential(*layers)

  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    #x = self.Tanh(x)
    x = self.maxpool(x)
    x = self.superblock2(x)
    x = self.superblock3(x)
    x = self.superblock4(x)
    x = self.superblock5(x)
    x = self.avgpool(x)
    x = torch.flatten(x, 1)
    #print(x.shape)
    #x = self.fc(x)
    x = self.fc1(x)
    x = self.fc2(x)
    return F.softmax(x, dim=1)
    #return x
    #return F.sigmoid(x)

  def init_weights(self):
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        m.weight.data.normal_(0, math.sqrt(2. / n))
      elif isinstance(m, nn.BatchNorm2d):
        m.weight.data.fill_(1)
        m.bias.data.zero_()

In [ ]:
 def build_resnet(backbone):
    if backbone == 'ResNet50':
      num_layers = [3, 4, 6, 3]
    elif backbone == 'ResNet101':
      num_layers = [3, 4, 23, 3]
    elif backbone == 'ResNet152':
      num_layers = [3, 8, 36, 3]
    elif backbone == 'ResNet18':
      num_layers = [2, 2, 2, 2]
    else:
      raise NotImplementedError

    model = ResNet(BuildingBlock, num_layers)
    return model

In [ ]:
def model(pretrained, requires_grad):
    model = models.resnet18(progress=True, pretrained=pretrained)
    #input layer of ResNet.....
    model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    # freeze hidden layers
    if requires_grad == False:
        for param in model.parameters():
            param.requires_grad = False
    # train the hidden layers
    elif requires_grad == True:
        for param in model.parameters():
            param.requires_grad = True
    # make the classification layer learnable
    numFcInputs = model.fc.in_features
    print(numFcInputs)
    model.fc = nn.Sequential(nn.Linear(numFcInputs, len(lb.classes_)), nn.Softmax(dim=1))
    return model

In [ ]:
#model = model(pretrained= True, requires_grad = False).to(device) #ResNet18 -- DeepModel
model = build_resnet(backbone='ResNet50').to(device)
#print(model)
print(summary(model, (1, 1024, 1024))) #, device = 'cpu'))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 512, 512]           3,136
       BatchNorm2d-2         [-1, 64, 512, 512]             128
              ReLU-3         [-1, 64, 512, 512]               0
         MaxPool2d-4         [-1, 64, 256, 256]               0
            Conv2d-5         [-1, 64, 256, 256]           4,096
       BatchNorm2d-6         [-1, 64, 256, 256]             128
              ReLU-7         [-1, 64, 256, 256]               0
            Conv2d-8         [-1, 64, 256, 256]          36,864
       BatchNorm2d-9         [-1, 64, 256, 256]             128
             ReLU-10         [-1, 64, 256, 256]               0
           Conv2d-11        [-1, 256, 256, 256]          16,384
      BatchNorm2d-12        [-1, 256, 256, 256]             512
           Conv2d-13        [-1, 256, 256, 256]          16,384
      BatchNorm2d-14        [-1, 256, 2

In [ ]:
#images, labels = next(iter(testloader))
#grid = torchvision.utils.make_grid(images*0.229+0.485 , nrow=4)   #0.485;   0.229,
#writer.add_image('image', grid)
#writer.add_graph(model, images.to(device).clone().detach())
#writer.close()

In [ ]:
#%load_ext tensorboard

In [ ]:
#tensorboard --logdir=runs

In [ ]:
#print(sum(p.numel() for p in model.parameters() if p.requires_grad))

In [ ]:
#for name, param in model.named_parameters():
#  print(name, param.data)

In [ ]:
def conMatrix(target, pred):
  nb_classes = 2   #confusion_matrix
  conf_Vector = []  #pred.view(-1), target.view(-1)
  conf_Vector = torch.tensor(conf_Vector, dtype=torch.long).to(device)
  conf_matrix = torch.zeros(nb_classes, nb_classes)
  for t, p in zip(target, pred):
    conf_matrix[t, p] += 1
  #conf_matrix =confusion_matrix(pred.view(-1), target.view(-1))
  for i in range(0, (2)):
    if i == 0:
      conf_Vector = torch.cat((conf_Vector, torch.diag(conf_matrix,i).to(device)), 0)
    else:
      conf_Vector = torch.cat((conf_Vector, torch.diag(conf_matrix,-i).to(device)), 0)
      conf_Vector = torch.cat((conf_Vector, torch.diag(conf_matrix,+i).to(device)), 0)
  confVector = (conf_Vector.data).cpu().numpy()
  print('    Evaluation Matrix')
  return confVector

In [ ]:
def L1_L2Reg(model):
  RegLoss = 0
  LamdaValue = 0.0 #1e-6
  for param in model.parameters():
    RegLoss += torch.sum(abs(param))
  return RegLoss*LamdaValue

In [ ]:
learnRate = 0.0001
criterion = nn.CrossEntropyLoss()  # loss function
WeightDecay = 0.000001 #0.001  #0.00005
# optimizer
optimizer = optim.Adam(model.parameters(), lr=learnRate, betas=(0.9, 0.999),
                       eps=1e-8,weight_decay=WeightDecay, amsgrad=False)

In [ ]:
# training function
def fit(model, train_dataloader, epoch):
    print('    Training.....', 'Epoch #', epoch)
    model.train()
    DataSize = 0
    totalItr=int(len(train_data)/train_dataloader.batch_size)
    train_running_loss = 0.0
    train_running_correct = 0
    for i, data in tqdm(enumerate(train_dataloader), total=int(len(train_data)/train_dataloader.batch_size)):
        data, target = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        DataSize+=target.size(0)  #print(DataSize)
        outputs = model.forward(data)
        error = criterion(outputs, target) #+ L1_L2Reg(model) #.unsqueeze(1))
        train_running_loss += (error.item()*target.size(0))
        #_, preds = torch.max(outputs.data, 1)  #torch.argmax(a, dim=1)
        #torch.argmax(a, dim=1)
        #preds = torch.argmax(F.softmax(outputs.data, dim=1), dim=1) #outputs.argmax(dim=1, keepdim=True)
        preds = torch.argmax(outputs.data, 1)
        #print(preds.dtype, target.dtype)
        train_running_correct += (preds == target).sum().item()
        #optimizer.zero_grad()
        loss = criterion(outputs, target) + L1_L2Reg(model)
        loss.backward()
        optimizer.step()
        del data, target, outputs, preds
        torch.cuda.empty_cache()

    train_loss = train_running_loss/DataSize #len(train_dataloader.dataset)
    train_accuracy = 100. * train_running_correct/DataSize #len(train_dataloader.dataset)
    #print(DataSize, totalItr*64, 100.*train_running_correct/DataSize, 100.*train_running_correct/(totalItr*64))
    #print(DataSize)
    #print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}")
    return train_loss, train_accuracy

In [ ]:
#validation function
def validate(model, test_dataloader):
    print('    Validating')
    model.eval()
    val_running_loss = 0.0   #i = torch.tensor([i]).float(), a = torch.cat((a, i), 0)
    val_running_correct = 0   #torch.tensor(label, dtype=torch.long)
    CofTarget, ConfPreds, ProbScores = [], [], []
    CofTarget = torch.tensor(CofTarget, dtype=torch.long).to(device)
    ConfPreds = torch.tensor(ConfPreds, dtype=torch.long).to(device)
    ProbScores = torch.tensor(ProbScores, dtype=torch.long).to(device)
    DataSize = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(test_dataloader), total=int(len(test_data)/test_dataloader.batch_size)):
            data, target = data[0].to(device), data[1].to(device)
            outputs = model.forward(data)
            loss = criterion(outputs, target) #.unsqueeze(1))
            DataSize += target.size(0)  # print(DataSize)
            val_running_loss += (loss.item()*target.size(0))
            #_, preds = torch.max(outputs.data, 1)
            #preds = outputs.argmax(dim=1, keepdim=True)  #tensor_array.cpu().detach().numpy()
            #preds = torch.argmax(F.softmax(outputs.data, dim=1), dim=1)
            preds = torch.argmax(outputs.data, 1)
            ProbScore = outputs.data[:,1]
            val_running_correct += (preds == target).sum().item()

            CofTarget = torch.cat((CofTarget, target), 0) #.to('cpu').numpy()]  #a = torch.cat((a, i), 0)
            ConfPreds = torch.cat((ConfPreds, preds), 0)  #[ConfPreds, preds.to('cpu').numpy()]
            ProbScores = torch.cat((ProbScores, ProbScore), 0)
            del data, target, outputs, preds, ProbScore
            torch.cuda.empty_cache()

        val_loss = val_running_loss/DataSize #len(test_dataloader.dataset)
        val_accuracy = 100. * val_running_correct/DataSize #len(test_dataloader.dataset)
        #print(CofTarget.size, ConfPreds.size)
        #return roc_auc_score(y_true, y_pred)
        #print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}')
        return val_loss, val_accuracy, CofTarget, ConfPreds, ProbScores

In [ ]:
#AUC-ROC evaluation....L1_L2Reg(model):
def ROC_AUC_Evaluation(CofTarget, ConfPreds, ProbScores):
  ConfVector = conMatrix(CofTarget, ConfPreds)
  Ytarget = CofTarget.cpu().numpy()
  Ypreds = ConfPreds.cpu().numpy()
  Yscore = ProbScores.cpu().numpy()
  RoC_Value = roc_auc_score(Ytarget,Yscore)
  FPRValue, TPRVlaue, _ = roc_curve(Ytarget, Yscore)
  return ConfVector, RoC_Value, FPRValue, TPRVlaue


In [ ]:
# lists to store per-epoch loss and accuracy values
train_loss, train_accuracy = [], []
val_loss, val_accuracy = [], []
ConfVector, RocAuC = [], []
fprValue, tprValue = [], []
start = time.time()
for epoch in range(1,11): #epochs):
    #print(f"Epoch {epoch+1} of {epochs}")

    #if np.remainder(epoch, 5) == 0:
      #train_data, trainloader = NewTrainLoader(BatchSize)
      #learnRate = alphaZero/(1+DecayRate*epoch)
      #optimizer = optim.Adam(model.parameters(), lr=learnRate, betas=(0.9, 0.999),
      #                      eps=1e-8, weight_decay=0.0000) #, amsgrad=True)

    train_epoch_loss, train_epoch_accuracy = fit(model, trainloader, epoch)
    writer.add_scalar("Loss/training", train_epoch_loss, epoch)

    val_epoch_loss, val_epoch_accuracy, CofTarget, ConfPreds, ProbScores = validate(model, testloader)
    writer.add_scalar("Loss/Validation", val_epoch_loss, epoch)

    #del train_data, trainloader
    #train_data, trainloader = NewTrainLoader(BatchSize)
    Conf_Vector, ROC_AUC, FPRValue, TPRValue = ROC_AUC_Evaluation(CofTarget, ConfPreds, ProbScores)
    #print(TPRVlaue.shape)
    RocAuC.append(ROC_AUC)
    fprValue.append(FPRValue)
    tprValue.append(TPRValue)
    ConfVector.append(Conf_Vector)
    train_loss.append(train_epoch_loss)
    train_accuracy.append(train_epoch_accuracy)
    val_loss.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)
    print(f"Train Loss: {train_epoch_loss:.4f}, Train Acc: {train_epoch_accuracy:.2f}")
    print(f'Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_accuracy:.2f}')
    print(Conf_Vector)
    print(ROC_AUC) #, TPRValue)

writer.flush()
end = time.time()
print(f"Training time: {(end-start)/60:.3f} minutes")

In [ ]:
print('Saving loss and accuracy plots...')
# accuracy plots
plt.figure(figsize=(10, 7))
plt.plot(train_accuracy, color='green', label='train accuracy')
plt.plot(val_accuracy, color='blue', label='validataion accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
#plt.savefig(f"../outputs/{acc_plot_name}.png")
plt.show()
# loss plots
plt.figure(figsize=(10, 7))
plt.plot(train_loss, color='orange', label='train loss')
plt.plot(val_loss, color='red', label='validataion loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
#plt.savefig(f"../outputs/{loss_plot_name}.png")
plt.show()

# serialize the model to disk
print('Saving model...')
#torch.save(model.state_dict(), f"../outputs/{model_name}.pth")

print('TRAINING COMPLETE')

In [ ]:
#print(RocAuC)
IndexRocAUC = np.argmax(RocAuC)
print(IndexRocAUC)

In [ ]:
#ROC-AUC plot
torch.set_printoptions(precision=2)
plt.figure(figsize=(10, 7))
plt.plot(FPRValue, TPRValue, label="ROC Curve, area = "+str('%.4f' % RocAuC[IndexRocAUC]))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.legend(loc=4)
plt.show()

In [ ]:
DtFrm = pd.DataFrame({'tAcc': train_accuracy, 'tLoss': train_loss, 'vAcc': val_accuracy, 'vLoss': val_loss})
DtFrm.to_csv('/content/drive/MyDrive/ResNet$18/ResNetSim#10.csv')

In [ ]:
DtFrmConf = pd.DataFrame (ConfVector,columns=['TP', 'TN', 'FP', 'FN'])
DtFrmConf.to_csv('/content/drive/MyDrive/ResNet$18/Confusion#10.csv') #test[:,0]

In [ ]:
# saving RocAuC,fprValue,tprVlaue
#DtFrmAuC = pd.DataFrame(RocAuC, tprValue, fprValue, columns=['RoCAuC','TPR', 'FPR'])
#DtFrmAuC.to_csv('/content/drive/MyDrive/ResNetModel$6/RoCAuC#70.csv')

In [ ]:
# save model checkpoint
torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(), 'loss': criterion,
            }, '/content/drive/MyDrive/ResNet$18/ResNet#10.pth')